# Setup

In this notebook, we set up the databases we need for the LCA. 

In [ ]:
import bw2data as bd
import bw2io as bi
import pandas as pd

In [ ]:
bd.projects.set_current("paper_plca_grid_expansion_rev")

## Creating background databases

For creating the background databases, we need access to the ecoinvent database (commercial) and to the default premise scenarios (free, to be asked from premise maintainers).

In [ ]:
ecoinvent_username = "xxx" # fill in your data here
ecoinvent_password = "xxx"
premise_key = "xxx" # to be asked from premise maintainers

### Getting ecoinvent

In [ ]:
bi.import_ecoinvent_release(
    version="3.10.1",
    system_model="cutoff",
    username=ecoinvent_username,
    password=ecoinvent_password,
)

### Creating prospective databases with `premise`

In [ ]:
from premise import *

In [ ]:
ndb = NewDatabase(
    scenarios=[
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2023},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2030},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2035},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2037},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2040},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2045},
        
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2023},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2025},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2030},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2035},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2037},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2040},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2045},
        
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2023},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2030},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2035},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2037},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2040},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2045},
    ],
    source_db="ecoinvent-3.10.1-cutoff",  # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10",  # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
    key=premise_key,  # to be requested from the library maintainers if you want to use default scenarios included in `premise`
    biosphere_name="ecoinvent-3.10.1-biosphere",  # <-- name of the biosphere database in the BW2 project. Must be a string.
)

ndb.update()

db_names = [
    "ei310_SSP2_PkBudg1000_2023",
    "ei310_SSP2_PkBudg1000_2025",
    "ei310_SSP2_PkBudg1000_2030",
    "ei310_SSP2_PkBudg1000_2035",
    "ei310_SSP2_PkBudg1000_2037",
    "ei310_SSP2_PkBudg1000_2040",
    "ei310_SSP2_PkBudg1000_2045",
    "ei310_SSP2_PkBudg650_2023",
    "ei310_SSP2_PkBudg650_2025",
    "ei310_SSP2_PkBudg650_2030",
    "ei310_SSP2_PkBudg650_2035",
    "ei310_SSP2_PkBudg650_2037",
    "ei310_SSP2_PkBudg650_2040",
    "ei310_SSP2_PkBudg650_2045",
    "ei310_SSP2_NPi_2023",
    "ei310_SSP2_NPi_2025",
    "ei310_SSP2_NPi_2030",
    "ei310_SSP2_NPi_2035",
    "ei310_SSP2_NPi_2037",
    "ei310_SSP2_NPi_2040",
    "ei310_SSP2_NPi_2045",
]


In [ ]:
ndb.write_db_to_brightway(db_names)

## Adding premise_gwp LCIA method

In [ ]:
from premise_gwp import add_premise_gwp
add_premise_gwp()

## Creating the grid databases

### Preparation

First, we load the background activities from ecoinvent. We write a function for this, as we need to do this for all our different background databases.

In [ ]:
def load_inputs_from_background(database_name: str):
    db = bd.Database(database_name)
    return {
        "glass_wool_mat": db.get(name="market for glass wool mat", location="GLO"),
        "porcelain": db.get(name="market for ceramic tile", location="GLO"),
        "epoxy": db.get(name="market for epoxy resin, liquid", location="RER"),
        "brass": db.get(name="market for brass", location="RoW"),
        "paint": db.get(name="market for electrostatic paint", location="GLO"),
        "paper": db.get(name="market for paper, melamine impregnated", location="RER"),
        "rubber": db.get(name="market for synthetic rubber", location="GLO"),
        "sulfur_hexafluoride": db.get(
            name="market for sulfur hexafluoride, liquid", location="RER"
        ),
        "lead": db.get(name="market for lead", location="GLO"),
        "bronze": db.get(name="market for bronze", location="GLO"),
        "polypropylene": db.get(
            name="market for polypropylene, granulate", location="GLO"
        ),
        "steel_sheet": db.get(name="market for sheet rolling, steel", location="GLO"),
        "steel_hot_rolled": db.get(
            name="market for steel, low-alloyed, hot rolled", location="GLO"
        ),
        "steel_lowalloyed": db.get(
            name="market for steel, low-alloyed", location="GLO"
        ),
        "steel_unalloyed": db.get(name="market for steel, unalloyed", location="GLO"),
        "transformer_oil": db.get(name="market for lubricating oil", location="RER"),
        "aluminium_wrought_alloy": db.get(
            name="market for aluminium, wrought alloy", location="GLO"
        ),
        "aluminium_sheet": db.get(
            name="market for sheet rolling, aluminium", location="GLO"
        ),
        "aluminium_cast": db.get(
            name="market for aluminium, cast alloy", location="GLO"
        ),
        "copper": db.get(name="market for copper, cathode", location="GLO"),
        "copper_wire": db.get(name="wire drawing, copper", location="RER"),
        "glass_fibre": db.get(name="market for glass fibre", location="GLO"),
        "kraft_paper": db.get(name="market for kraft paper", location="RER"),
        "softwood": db.get(
            name="market for sawnwood, softwood, raw, dried (u=20%)", location="RER"
        ),
        "concrete": db.get(
            name="market for concrete, normal strength", location="RoW"
        ),  # density of 2200 kg/m3 assumed for conversion, see Itten et al.
        "zinc": db.get(name="market for zinc", location="GLO"),
        "waste_concrete": db.get(
            name="market for waste concrete", location="Europe without Switzerland"
        ),
        "waste_polyethylene": db.get(
            name="market for waste polyethylene", location="DE"
        ),
        "waste_oil": db.get(
            name="market for waste mineral oil", location="Europe without Switzerland"
        ),
        "excavation": db.get(
            name="market for excavation, hydraulic digger", location="GLO"
        ),
        "polyester_resin": db.get(
            name="market for polyester resin, unsaturated", location="RER"
        ),
        "steel_chromium_steel": db.get(
            name="market for steel, chromium steel 18/8", location="GLO"
        ),
        "polycarbonate": db.get(name="market for polycarbonate", location="RER"),
        "wood": db.get(name="market for fibreboard, hard", location="RER"),
        "cement_unspecified": db.get(
            name="market for cement, unspecified", location="Europe without Switzerland"
        ),
        "glass_tube_borosilicate": db.get(
            name="market for glass tube, borosilicate", location="GLO"
        ),
        "extrusion_plastic_pipes": db.get(
            name="market for extrusion, plastic pipes", location="GLO"
        ),
        "polyethylene": db.get(
            name="market for polyethylene, low density, granulate", location="GLO"
        ),
        "copper_wire_drawing": db.get(
            name="market for wire drawing, copper", location="GLO"
        ),
    }


Metadata for the grid components:

In [ ]:
# (code, name, unit) of grid components
COMPONENT_INFO = [
    ("disconnector", "Disconnector", "unit"),
    (
        "land_cable_oil_cu_150kv",
        "Land cable, oil insulated, copper, 150kV",
        "kilometer",
    ),
    ("overhead_line_400kv", "Overhead line, 400kV", "kilometer"),
    ("overhead_line_150kv", "Overhead line, 150kV", "kilometer"),
    ("overhead_line_10kv", "Overhead line, 10kV", "kilometer"),
    ("overhead_line_04kv", "Overhead line, 0.4kV", "kilometer"),
    ("overhead_line_HVDC", "Overhead line HVDC", "kilometer"),
    ("land_cable_oil_cu_HVDC", "Land cable, oil insulated, copper, HVDC", "kilometer"),
    ("transformer_250mva", "Transformer, 250MVA", "unit"),
    ("transformer_40mva", "Transformer, 40MVA", "unit"),
    ("transformer_315kva", "Transformer, 315kVA", "unit"),
    (
        "land_cable_vpe_al_04kv",
        "Land cable, vpe insulated, aluminium, 0.4kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    ("land_cable_epr_cu_11kv", "Land cable, epr insulated, copper, 11kV", "kilometer"),
    (
        "land_cable_vpe_al_50kv",
        "Land cable, vpe insulated, aluminium, 50kV",
        "kilometer",
    ),
    ("land_cable_vpe_cu_1kv", "Land cable, vpe insulated, copper, 1kV", "kilometer"),
    ("gas_insulated_switchgear_420kv", "Gas insulated switchgear, 420kV", "unit"),
    ("substation_lv", "Substation, LV", "unit"),
    ("substation_mv", "Substation, MV", "unit"),
]

A function to help create the grid component activities:

In [ ]:

def create_component_nodes(base_database_name: str, component_database_name: str, reset: bool=True):
    if reset:
        if component_database_name in bd.databases:
            del bd.databases[component_database_name]
        db_components = bd.Database(component_database_name)
        db_components.register()
    else :
        db_components = bd.Database(component_database_name)
    
    INPUT_NODES = load_inputs_from_background(base_database_name)
    
    for code, name, unit in COMPONENT_INFO:
        try:
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
        except:
            db_components.get(code).delete()
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
            component.save()
        component.save()
        component.new_edge(
            name=component["name"], input=component, amount=1, unit=unit, type="production"
        ).save()

        inputs_df = pd.read_csv(f"data/lci/{code}.csv", sep=";", index_col="input")
        for input_code, amount, unit in zip(inputs_df.index, inputs_df.amount, inputs_df.unit):
            component.new_edge(
                input=INPUT_NODES[input_code], amount=amount, type="technosphere"
            ).save()

        if code == "gas_insulated_switchgear_420kv":
            sf6_node = bd.get_node(database="ecoinvent-3.10.1-biosphere", name="Sulfur hexafluoride", categories=("air",))
            component.new_edge(
                input=sf6_node,
                amount=28.6, # leakage
                type="biosphere",
            ).save()

Finally, we need the grid expansion numbers:

In [ ]:
import bw2data as bd

# Base share values (central estimates)
SHARE_DEFAULTS = {
    "oh": {  # Overhead lines
        "ehv_ac": 1.0,
        "ehv_dc": 0.05,
        "hv": 0.9474,
        "mv": 0.1847,
        "lv": 0.0652,
    },
    "al": {  # Aluminum cables
        "ehv": 0.0,
        "hv": 0.5,
        "mv": 0.75,
        "lv": 0.75,
    }
}

# Total infrastructure quantities per BASE year (km or units)
# Note: Interpolated years (2025, 2030, 2035, 2040) get their totals from distributed_components
TOTAL_QUANTITIES = {
    2023: {
        "ehv_ac": 36400, "ehv_dc": 2172, "hv": 95200, "mv": 530200, "lv": 1570100,
        "transformer_250mva": 709, "transformer_40mva": 7278, "transformer_315kva": 577790,
        "gas_insulated_switchgear_420kv": 7278, "substation_mv": 7278 + 709, "substation_lv": 577790,
        "land_cable_epr_cu_11kv": 117593,
    },
    2037: {
        "ehv_ac": 11823, "ehv_dc": 17492, "hv": 28307, "mv": 117593, "lv": 375511,
        "transformer_250mva": 221, "transformer_40mva": 1890, "transformer_315kva": 133273,
        "gas_insulated_switchgear_420kv": 1890, "substation_mv": (1890+709)/1.58, "substation_lv": 133273,
        "land_cable_epr_cu_11kv": 117593,
    },
    2045: {
        "ehv_ac": 250, "ehv_dc": 4683, "hv": 15096, "mv": 64837, "lv": 198933,
        "transformer_250mva": 59, "transformer_40mva": 1022, "transformer_315kva": 71943,
        "gas_insulated_switchgear_420kv": 1022, "substation_mv": (1022+709)/1.58, "substation_lv": 71943,
        "land_cable_epr_cu_11kv": 117593,
    },
}

# Component to voltage level mapping for parameter lookup
# Format: component_key -> (param_type, level)
# param_type: "oh" = overhead line, "oh_al" = underground (aluminum or copper)
COMPONENT_LEVEL_MAP = {
    "overhead_line_400kv": ("oh", "ehv_ac"),
    "overhead_line_HVDC": ("oh", "ehv_dc"),
    "overhead_line_150kv": ("oh", "hv"),
    "overhead_line_10kv": ("oh", "mv"),
    "overhead_line_04kv": ("oh", "lv"),
    "land_cable_oil_cu_HVDC": ("ug_cu", "ehv_dc"),  # underground copper (HVDC)
    "land_cable_vpe_al_50kv": ("ug_al", "hv"),      # underground aluminum
    "land_cable_oil_cu_150kv": ("ug_cu", "hv"),    # underground copper
    "land_cable_vpe_al_10kv": ("ug_al", "mv"),
    "land_cable_epr_cu_11kv": ("ug_cu", "mv"),
    "land_cable_vpe_al_04kv": ("ug_al", "lv"),
    "land_cable_vpe_cu_1kv": ("ug_cu", "lv"),
}

def calculate_uncertainty_range(year):
    """
    Calculate the percentage deviation for uncertainty bounds.
    Progressive uncertainty: 5% in 2023, linearly increasing to 20% by 2045.
    """
    min_year, max_year = 2023, 2045
    min_uncertainty, max_uncertainty = 0.05, 0.20
    
    # Clamp year to range
    year = max(min_year, min(max_year, year))
    
    # Linear interpolation
    progress = (year - min_year) / (max_year - min_year)
    return min_uncertainty + progress * (max_uncertainty - min_uncertainty)


def setup_project_parameters(years):
    """
    Registers year-specific parameters in the Brightway project.
    Uses triangular distribution (uncertainty_type=5) with progressive uncertainty.
    """
    params = []
    
    for year in years:
        uncertainty_pct = calculate_uncertainty_range(year)
        
        # Overhead Line Share parameters
        for lvl, val in SHARE_DEFAULTS["oh"].items():
            if val == 1.0:
                minimum = max(0.0, val - uncertainty_pct)
                maximum = 1.0
            elif val == 0.0:
                minimum = 0.0
                maximum = min(1.0, val + uncertainty_pct)
            else:
                minimum = max(0.0, val * (1 - uncertainty_pct))
                maximum = min(1.0, val * (1 + uncertainty_pct))
            
            params.append({
                "name": f"share_oh_{lvl}_{year}",
                "amount": val,
                "uncertainty type": 5,  # Triangular distribution
                "loc": val,
                "minimum": minimum,
                "maximum": maximum,
            })
        
        # Aluminum Cable Share parameters
        for lvl, val in SHARE_DEFAULTS["al"].items():
            if val == 1.0:
                minimum = max(0.0, val - uncertainty_pct)
                maximum = 1.0
            elif val == 0.0:
                minimum = 0.0
                maximum = min(1.0, val + uncertainty_pct)
            else:
                minimum = max(0.0, val * (1 - uncertainty_pct))
                maximum = min(1.0, val * (1 + uncertainty_pct))
            
            params.append({
                "name": f"share_al_{lvl}_{year}",
                "amount": val,
                "uncertainty type": 5,
                "loc": val,
                "minimum": minimum,
                "maximum": maximum,
            })
    
    # Register parameters in the project
    bd.parameters.new_project_parameters(params)
    print(f"Registered {len(params)} parameters for years: {years}")
    return params


def get_base_total_for_component(component, current_value, year):
    """
    Calculate the base total (before share multiplication) for a component.
    This reverses the share calculation to get the original total.
    
    For interpolated years, we use the current_value and reverse-engineer the total.
    """
    if component not in COMPONENT_LEVEL_MAP:
        return None
    
    param_type, level = COMPONENT_LEVEL_MAP[component]
    oh_share = SHARE_DEFAULTS["oh"].get(level, 0)
    al_level = "ehv" if level == "ehv_dc" else level
    al_share = SHARE_DEFAULTS["al"].get(al_level, 0)
    
    # Reverse the share calculation to get the base total
    if param_type == "oh":
        # current_value = oh_share * total
        if oh_share > 0:
            return current_value / oh_share
        return current_value
    elif param_type == "ug_cu":
        # For HVDC: current_value = (1 - oh_share) * total
        # For others: current_value = (1 - oh_share) * (1 - al_share) * total
        if level == "ehv_dc":
            if (1 - oh_share) > 0:
                return current_value / (1 - oh_share)
        else:
            divisor = (1 - oh_share) * (1 - al_share)
            if divisor > 0:
                return current_value / divisor
        return current_value
    elif param_type == "ug_al":
        # current_value = (1 - oh_share) * al_share * total
        divisor = (1 - oh_share) * al_share
        if divisor > 0:
            return current_value / divisor
        return current_value
    
    return None


def get_formula_for_component(component, year, current_value):
    """
    Constructs the formula string for an exchange based on the component type.
    
    Args:
        component: The component key (e.g., "overhead_line_400kv")
        year: The year for parameter lookup
        current_value: The pre-computed value from distributed_components
    
    Logic:
    - Overhead Line: share_oh * total
    - Al Cable: (1 - share_oh) * share_al * total  
    - Cu Cable: (1 - share_oh) * (1 - share_al) * total
    """
    if component not in COMPONENT_LEVEL_MAP:
        return None  # Fixed components (transformers, substations)
    
    param_type, level = COMPONENT_LEVEL_MAP[component]
    
    # Calculate the base total from the current value
    total = get_base_total_for_component(component, current_value, year)
    
    if total is None or total == 0:
        return None
    
    # Map level names for AL parameters (ehv_dc uses 'ehv')
    al_level = "ehv" if level == "ehv_dc" else level
    
    if param_type == "oh":
        # Pure overhead line: share_oh * total
        return f"share_oh_{level}_{year} * {total}"
    
    elif param_type == "ug_cu":
        if level == "ehv_dc":
            # HVDC underground: (1 - share_oh) * total (no aluminum option)
            return f"(1 - share_oh_{level}_{year}) * {total}"
        else:
            # Copper cable: (1 - share_oh) * (1 - share_al) * total
            return f"(1 - share_oh_{level}_{year}) * (1 - share_al_{al_level}_{year}) * {total}"
    
    elif param_type == "ug_al":
        # Aluminum cable: (1 - share_oh) * share_al * total
        return f"(1 - share_oh_{level}_{year}) * share_al_{al_level}_{year} * {total}"
    
    return None


# Build grid_compositions dictionary with computed values
# (These are the deterministic values; formulas will be applied to exchanges)
grid_compositions = {}

for year in [2023, 2037, 2045]:
    q = TOTAL_QUANTITIES[year]
    oh = SHARE_DEFAULTS["oh"]
    al = SHARE_DEFAULTS["al"]
    
    grid_compositions[year] = {
        # EHV AC
        "overhead_line_400kv": oh["ehv_ac"] * q["ehv_ac"],
        # EHV DC
        "overhead_line_HVDC": oh["ehv_dc"] * q["ehv_dc"],
        "land_cable_oil_cu_HVDC": (1 - oh["ehv_dc"]) * q["ehv_dc"],
        # HV
        "overhead_line_150kv": oh["hv"] * q["hv"],
        "land_cable_vpe_al_50kv": (1 - oh["hv"]) * al["hv"] * q["hv"],
        "land_cable_oil_cu_150kv": (1 - oh["hv"]) * (1 - al["hv"]) * q["hv"],
        "transformer_250mva": q["transformer_250mva"],
        # MV
        "overhead_line_10kv": oh["mv"] * q["mv"],
        "land_cable_vpe_al_10kv": (1 - oh["mv"]) * al["mv"] * q["mv"],
        "land_cable_epr_cu_11kv": (1 - oh["mv"]) * (1 - al["mv"]) * q["land_cable_epr_cu_11kv"],
        "transformer_40mva": q["transformer_40mva"],
        "gas_insulated_switchgear_420kv": q["gas_insulated_switchgear_420kv"],
        "substation_mv": q["substation_mv"],
        # LV
        "overhead_line_04kv": oh["lv"] * q["lv"],
        "land_cable_vpe_al_04kv": (1 - oh["lv"]) * al["lv"] * q["lv"],
        "land_cable_vpe_cu_1kv": (1 - oh["lv"]) * (1 - al["lv"]) * q["lv"],
        "transformer_315kva": q["transformer_315kva"],
        "substation_lv": q["substation_lv"],
    }

print("Grid compositions computed successfully.")
print(f"Uncertainty range: 5% in 2023 -> 20% in 2045 (triangular distribution)")

### Creating grid nodes

#### Grid status quo

In [ ]:
db_status_quo = bd.Database("grid_status_quo")
background_2023 = bd.Database("ei310_SSP2_NPi_2023") # common basis: ssp2 base scenario

create_component_nodes(background_2023.name, db_status_quo.name)

node_code = 'grid_status_quo'
node_name = node_code

try:
    grid = db_status_quo.new_node(code=node_code, name=node_name)
except:
    existing_node = db_status_quo.get(node_code)
    existing_node.delete()
    grid = db_status_quo.new_node(code=node_code, name=node_name)
grid.save()
grid.new_edge(input=grid, amount=1, type='production').save()
for key, value in grid_compositions[2023].items():
    grid.new_edge(input=db_status_quo.get(key), amount=value, type='technosphere').save()

Aggregated material nodes for status quo, needed for the sankey diagram.

In [ ]:
material_exchanges = {
    'aluminium': {},
    'steel': {},
    'concrete': {},
    'copper': {},
    'plastics': {}
}

grid = bd.get_node(code="grid_status_quo")
for component_exchange in grid.technosphere():
    for material_exchange in component_exchange.input.technosphere():
        name = material_exchange.input['name'].lower()
        key = None
        if "aluminium" in name:
            key = 'aluminium'
        elif "steel" in name or "iron" in name:
            key = 'steel'
        elif "concrete" in name:
            key = 'concrete'
        elif "copper" in name:
            key = 'copper'
        elif "polyethylene" in name or "polypropylene" in name or "plastic" in name:
            key = 'plastics'

        total_amount = material_exchange.amount * component_exchange.amount
        
        if key:
            if material_exchange.input in material_exchanges[key]:
                material_exchanges[key][material_exchange.input] += total_amount
            else:
                material_exchanges[key][material_exchange.input] = total_amount
        else:
            if material_exchange.input in material_exchanges.setdefault('other', {}):
                material_exchanges['other'][material_exchange.input] += total_amount
            else:
                material_exchanges['other'][material_exchange.input] = total_amount


In [ ]:
aggregated_material_nodes = []
for name, subexchanges in material_exchanges.items():
    try:
        mat = db_status_quo.new_node(
            name=f"aggregated material: {name}"
        )
    except:
        db_status_quo.get(name).delete()
        mat = db_status_quo.new_node(
            name=f"aggregated material: {name}"
        )
    mat.save()
    aggregated_material_nodes.append(mat)
    mat.new_exchange(input=mat, amount=1, type="production").save()
    for node, value in subexchanges.items():
        mat.new_exchange(input=node, amount=value, type="technosphere").save()

### Grid expansion nodes

Distributing grid expansion periods to sub-expansion periods that can be linked to prospective databases

In [ ]:
# Current year
current_year = 2023
expansion_period_1 = 2037 - current_year
expansion_period_2 = 2045 - 2037

# Target years for the distribution
years_2037 = [2023, 2025, 2030, 2035, 2037]
timespan_2037 = years_2037[-1] - years_2037[0]
years_2045 = [2037, 2040, 2045]
timespan_2045 = years_2045[-1] - years_2045[0]

distributed_components = {}
for index, year in enumerate(years_2037):
    if year < 2037:
        year_data = {}
        duration = years_2037[index+1] - year 
        factor = duration / timespan_2037
        
        year_data = {}
        for component, total_value in grid_compositions[2037].items():
            year_data[component] = total_value * factor
        distributed_components[year] = year_data
        
for index, year in enumerate(years_2045):
    if year < 2045:
        year_data = {}
        duration = years_2045[index+1] - year 
        factor = duration / timespan_2045
        
        year_data = {}
        for component, total_value in grid_compositions[2045].items():
            year_data[component] = total_value * factor
        distributed_components[year] = year_data

Saving the results for later use

In [ ]:
import json

json.dump(distributed_components, open("data/results/distributed_components.json", "w"))

Making sure the distributed numbers sum up correctly:

In [ ]:
def test_distributed_sums(components, distributed_data):
    original_totals = {}
    for year in components:
        if year != 2023:
            for component, value in components[year].items():
                original_totals[component] = original_totals.get(component, 0) + value

    distributed_sums = {component: 0 for component in original_totals.keys()}
    for year in distributed_data.values():
        for component, value in year.items():
            distributed_sums[component] += value

    all_passed = True
    for component, total in original_totals.items():
        if not round(distributed_sums[component], 1) == round(total, 1):
            print(f"Test failed for {component}: distributed sum {distributed_sums[component]} != original total {total}")
            all_passed = False
    assert all_passed

test_result = test_distributed_sums(grid_compositions, distributed_components)

To link the prospective grid activities to the correct background databases, we need the info what database represents what year:

In [ ]:
background_db_time_mapping = {
    'ei310_SSP2_NPi_2023': 2023,
    'ei310_SSP2_NPi_2025': 2025,
    'ei310_SSP2_NPi_2030': 2030,
    'ei310_SSP2_NPi_2035': 2035,
    'ei310_SSP2_NPi_2037': 2037,
    'ei310_SSP2_NPi_2040': 2040,
    #    
    'ei310_SSP2_PkBudg1000_2023': 2023,
    'ei310_SSP2_PkBudg1000_2025': 2025,
    'ei310_SSP2_PkBudg1000_2030': 2030,
    'ei310_SSP2_PkBudg1000_2035': 2035,
    'ei310_SSP2_PkBudg1000_2037': 2037,
    'ei310_SSP2_PkBudg1000_2040': 2040,
    #
    'ei310_SSP2_PkBudg650_2023': 2023,
    'ei310_SSP2_PkBudg650_2025': 2025,
    'ei310_SSP2_PkBudg650_2030': 2030,
    'ei310_SSP2_PkBudg650_2035': 2035,
    'ei310_SSP2_PkBudg650_2037': 2037,
    'ei310_SSP2_PkBudg650_2040': 2040,
}

ends_of_expansion_periods = {
    2023: 2025,
    2025: 2030,
    2030: 2035,
    2035: 2037,
    2037: 2040,
    2040: 2045,
}

Creating the prospective grid activities:

In [ ]:
# --- Setup Parameters First ---
# Get all unique years from the mapping
all_years = sorted(set(background_db_time_mapping.values()))
setup_project_parameters(all_years)

# --- Clean up old parameter groups ---
from bw2data.parameters import Group, ActivityParameter, ParameterizedExchange

# Remove old/stale parameter groups that might cause conflicts
old_groups = ["grid_share_parameters", "grid_expansion_prospective_params", "grid_expansion_static_params"]
for group_name in old_groups:
    try:
        group = Group.get(name=group_name)
        # Delete associated activity parameters
        ActivityParameter.delete().where(ActivityParameter.group == group_name).execute()
        # Delete associated parameterized exchanges
        ParameterizedExchange.delete().where(ParameterizedExchange.group == group_name).execute()
        # Delete the group
        group.delete_instance()
        print(f"Cleaned up old group: {group_name}")
    except:
        pass  # Group doesn't exist, that's fine

# --- Create Grid Expansion Database with Parameterized Exchanges ---
nodes_prospective_expansion = []
db_expansion = bd.Database("grid_expansion_prospective")
if db_expansion.name in bd.databases:
    del bd.databases[db_expansion.name]
db_expansion.register()

# Use database-specific parameter group
param_group_prospective = "grid_expansion_prospective_params"

for db_name, year in background_db_time_mapping.items():
    scenario = db_name.split("_")[2]
    db_components = bd.Database(f"grid_components_{scenario}_{year}")
    if year != 2023:
        db_background = bd.Database(db_name)
    else:
        db_background = bd.Database("ei310_SSP2_NPi_2023") # choose same basis for status quo

    create_component_nodes(db_background.name, db_components.name)

    node_code = f"grid_{scenario}_{ends_of_expansion_periods[year]}"
    node_name = node_code

    try:
        grid = db_expansion.new_node(code=node_code, name=node_name)
    except:
        existing_node = db_expansion.get(node_code)
        existing_node.delete()
        grid = db_expansion.new_node(code=node_code, name=node_name)
    grid.save()

    nodes_prospective_expansion.append(grid)

    grid.new_edge(input=grid, amount=1, type="production").save()
    
    # Create exchanges with formulas for parameterized components
    for key, value in distributed_components[year].items():
        formula = get_formula_for_component(key, year, value)
        
        if formula:
            # Create exchange with formula for parameterized components
            edge = grid.new_edge(
                input=db_components.get(key), 
                amount=value,  # Default amount
                type="technosphere",
                formula=formula,
            )
        else:
            # Fixed components (transformers, substations) - no formula
            edge = grid.new_edge(
                input=db_components.get(key), 
                amount=value, 
                type="technosphere"
            )
        edge.save()
    
    # Add this activity to the database-specific parameter group
    bd.parameters.add_exchanges_to_group(param_group_prospective, grid)

# Recalculate only the specific group we just created
ActivityParameter.recalculate(param_group_prospective)
print(f"Created {len(nodes_prospective_expansion)} grid expansion nodes with parameterized exchanges.")

Static expansion for comparison:

In [ ]:
# Static expansion for comparison (also parameterized)
nodes_static_expansion = []

db_static_expansion = bd.Database(f"grid_expansion_static")
if db_static_expansion.name in bd.databases:
    del bd.databases[db_static_expansion.name]
db_static_expansion.register()

# Use database-specific parameter group
param_group_static = "grid_expansion_static_params"

years = [
    2023,
    2025,
    2030,
    2035,
    2037,
    2040,
]

for year in years:
    node_code = f"grid_static_{ends_of_expansion_periods[year]}"
    node_name = node_code
    try:
        grid = db_static_expansion.new_node(code=node_code, name=node_name)
    except:
        existing_node = db_static_expansion.get(node_code)
        existing_node.delete()
        grid = db_static_expansion.new_node(code=node_code, name=node_name)
    grid.save()

    nodes_static_expansion.append(grid)

    grid.new_edge(input=grid, amount=1, type="production").save()
    
    for key, value in distributed_components[year].items():
        formula = get_formula_for_component(key, year, value)
        
        if formula:
            # Create exchange with formula for parameterized components
            edge = grid.new_edge(
                input=db_status_quo.get(key), 
                amount=value,
                type="technosphere",
                formula=formula,
            )
        else:
            # Fixed components - no formula
            edge = grid.new_edge(
                input=db_status_quo.get(key), 
                amount=value, 
                type="technosphere"
            )
        edge.save()
    
    # Add to database-specific parameter group
    bd.parameters.add_exchanges_to_group(param_group_static, grid)

# Recalculate only this specific group
ActivityParameter.recalculate(param_group_static)
print(f"Created {len(nodes_static_expansion)} static expansion nodes with parameterized exchanges.")

### Add electricity mixes for comparison of emissions per kWh electricity

We base the assessment on the German market for electricity, low voltage and remove all grid-related exchanges.

In [ ]:
old_mix_lv_2023 = bd.get_node(database="ei310_SSP2_NPi_2023", name="market group for electricity, low voltage", location="DEU")

In [ ]:
old_mix_mv_2023 = bd.get_node(database="ei310_SSP2_NPi_2023", name="market group for electricity, low voltage", location="DEU")

In [ ]:
if "new_mixes" in bd.databases:
    del bd.databases["new_mixes"]
db_new_mixes = bd.Database("new_mixes")
db_new_mixes.register()

dbs = ["ei310_SSP2_NPi_2023", "ei310_SSP2_NPi_2045", "ei310_SSP2_PkBudg1000_2045", "ei310_SSP2_PkBudg650_2045"]

for db, scenario, year in zip(dbs, ["SS2_NPi", "SSP2_NPi", "SSP2-PkBudg1000", "SSP2-PkBudg650"], [2023, 2045, 2045, 2045]):
    # MV LEVEL
    old_mix_mv_2023 = bd.get_node(database=db, name="market group for electricity, medium voltage", location="DEU")
    activity_mv_name = f'market group for electricity, medium voltage DE {year} {scenario} NO GRID'
    activity_mv_code = f'el_mv_{year}_DE_{scenario}'

    try:
        new_mix_mv = db_new_mixes.new_node(code=activity_mv_code, name=activity_mv_name)
    except:
        existing_activity = db_new_mixes.get(activity_mv_code)
        existing_activity.delete()
        new_mix_mv = db_new_mixes.new_activity(code=activity_mv_code, name=activity_mv_name, unit='kilowatt hour', location="DEU", **{'reference product': activity_mv_name})
    new_mix_mv.save()

    for exc in old_mix_mv_2023.exchanges():
        if 'transmission network' in exc['name'] or 'sulfur hexafluoride' in exc['name'].lower():
            continue
        if exc['type'] == 'production':
            new_mix_mv.new_edge(input=new_mix_mv.key, amount=1, type='production').save()
        elif exc["name"] == "market group for electricity, medium voltage":
            new_mix_mv.new_exchange(input=new_mix_mv, amount=exc['amount'], type=exc['type']).save()
        else:
            new_mix_mv.new_exchange(input=exc['input'], amount=exc['amount'], type=exc['type']).save()

    # LV LEVEL
    old_mix_lv = bd.get_node(database=db, name="market group for electricity, low voltage", location="DEU") 
    activity_lv_name = f'market group for electricity, low voltage DE {year} {scenario} NO GRID'
    activity_lv_code = f'el_lv_{year}_DE_{scenario}'

    try:
        new_mix_lv = db_new_mixes.new_node(code=activity_lv_code, name=activity_lv_name)
    except:
        existing_activity = db_new_mixes.get(activity_lv_code)
        existing_activity.delete()
        new_mix_lv = db_new_mixes.new_activity(code=activity_lv_code, name=activity_lv_name, unit='kilowatt hour', location="DEU", **{'reference product': activity_lv_name})
    new_mix_lv.save()

    for exc in old_mix_lv.exchanges():
        if 'distribution network' in exc['name'] or 'sulfur hexafluoride' in exc['name'].lower():
            continue
        if exc['type'] == 'production':
            new_mix_lv.new_edge(input=new_mix_lv.key, amount=1, type='production').save()
        elif exc["name"] == "market group for electricity, low voltage":
            new_mix_lv.new_exchange(input=new_mix_lv, amount=exc['amount'], type=exc['type']).save()
        elif exc["name"] == "market group for electricity, medium voltage":
            new_mix_lv.new_exchange(input=new_mix_mv, amount=exc['amount'], type=exc['type']).save()
        else:
            new_mix_lv.new_exchange(input=exc['input'], amount=exc['amount'], type=exc['type']).save()

### Add additional scenario for recycled material shares in Aluminium Production

In [ ]:

shares_secondary_alu = { # from IAI Report "Aluminium Sector Greenhouse Gas Pathways to 2050"
    "SSP2-PkBudg650": { # 1.5°C scenario from original report
        2018: 0.33,
        2030: 0.43,
        2035: 0.45,
        2040: 0.48,
        2045: 0.51,
    },
    "SSP2-PkBudg1000": { # <2°C scenario from original report
        2018: 0.33,
        2030: 0.44,
        2035: 0.47,
        2040: 0.48,
        2045: 0.50,
    },
    "SSP2-NPi": { # BAU scenario from original report
        2018: 0.33,
        2030: 0.42,
        2035: 0.44,
        2040: 0.45,
        2045: 0.46,
    }
}

def extract_scenario_from_db_name(db_name: str) -> str:
    parts = db_name.split("_")
    scenario_part = parts[1]  # e.g., "SSP2"
    scenario_type_part = parts[2]  # e.g., "NPi", "PkBudg1000", "PkBudg650"
    return f"{scenario_part}-{scenario_type_part}"

def interpolate_scrap_share(year: int, scenario: str) -> float:
    shares = shares_secondary_alu[scenario]
    years = sorted(shares.keys()) 
    
    if year in shares:
        return shares[year]
    
    for i in range(len(years) - 1):
        if years[i] < year < years[i + 1]:
            year_start = years[i]
            year_end = years[i + 1]
            share_start = shares[year_start]
            share_end = shares[year_end]
            # Linear interpolation
            interpolated_share = share_start + (share_end - share_start) * ((year - year_start) / (year_end - year_start))
            return interpolated_share
    

for db_name, year in background_db_time_mapping.items():    
    # 1. Group the exchanges
    alu = bd.Database(db_name).get(name="market for aluminium, wrought alloy", location="GLO")
    scrap_exchanges = [exc for exc in alu.technosphere() if "treatment of aluminium scrap" in exc.input['name'].lower()]
    primary_exchanges = [exc for exc in alu.technosphere() if "aluminium ingot, primary" in exc.input['name'].lower()]
    
    # 2. Get current sub-totals to determine internal proportions
    current_scrap_total = sum(exc.amount for exc in scrap_exchanges)
    current_primary_total = sum(exc.amount for exc in primary_exchanges)
    
    # 3. Define the new target shares (Target Total = 1)
    new_scrap_share = interpolate_scrap_share(year=year, scenario=extract_scenario_from_db_name(db_name))
    new_primary_share = 1 - new_scrap_share
    
    # 4. Scale Scrap Exchanges
    if current_scrap_total > 0:
        for exc in scrap_exchanges:
            proportion = exc.amount / current_scrap_total
            new_amount = proportion * new_scrap_share
            print(f"  [Scrap] Scaling {exc.input['location']} {exc.input['name'][:30]}...: {exc.amount:.4f} -> {new_amount:.4f}")
            exc["amount"] = new_amount
            exc.save()

    # 5. Scale Primary Exchanges
    if current_primary_total > 0:
        for exc in primary_exchanges:
            proportion = exc.amount / current_primary_total
            new_amount = proportion * new_primary_share
            print(f"  [Prim] Scaling {exc.input['location']} {exc.input['name'][:30]}...: {exc.amount:.4f} -> {new_amount:.4f}")
            exc["amount"] = new_amount
            exc.save()

## Aggregating materials for faster monte carlo runs

In [ ]:
agg_dbs = [db for db in bd.databases if db.startswith("agg_")]

In [ ]:
for db in agg_dbs:
    del bd.databases[db]

In [ ]:
import bw2calc as bc

method = ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')
co2 = bd.get_node(database="ecoinvent-3.10.1-biosphere", name="Carbon dioxide, fossil", categories=("air",))

for db_name in background_db_time_mapping.keys():
    bg_inputs = list(load_inputs_from_background(db_name).values())
    new_db = bd.Database("agg_" + db_name)
    new_db.register()
    lca = bc.LCA({bg_inputs[0]: 1}, method=method)
    lca.lci(factorize=True)
    for inp in bg_inputs:
        lca.redo_lcia(demand={inp.id: 1})
        agg_act = new_db.new_node(name="agg" + inp['name'])
        agg_act.save()
        agg_act.new_edge(input=agg_act, amount=1, type="production").save()
        agg_act.new_edge(input=co2, amount=lca.score, type="biosphere").save()
        

In [ ]:
comp_dbs = [ 
 'grid_components_NPi_2023',
 'grid_components_NPi_2025',
 'grid_components_NPi_2030',
 'grid_components_NPi_2035',
 'grid_components_NPi_2037',
 'grid_components_NPi_2040',
 'grid_components_PkBudg1000_2023',
 'grid_components_PkBudg1000_2025',
 'grid_components_PkBudg1000_2030',
 'grid_components_PkBudg1000_2035',
 'grid_components_PkBudg1000_2037',
 'grid_components_PkBudg1000_2040',
 'grid_components_PkBudg650_2023',
 'grid_components_PkBudg650_2025',
 'grid_components_PkBudg650_2030',
 'grid_components_PkBudg650_2035',
 'grid_components_PkBudg650_2037',
 'grid_components_PkBudg650_2040',]

# for db in comp_dbs:
#     del bd.databases["agg_" + db]
    
for db_name in comp_dbs:
    new_db = bd.Database("agg_" + db_name)
    new_db.register()
    for act in bd.Database(db_name):
        try:
            new_db.get(name="agg" + act['name'])
            print(f"Aggregated activity for {act['name'], db_name} already exists, skipping...")
            continue
        except:
            pass
        new_act = new_db.new_node(name="agg" + act['name'])
        new_act.save()
        new_act.new_edge(input=new_act, amount=1, type="production").save()
        for exc in act.technosphere():
            try:
                agg_input = bd.get_node(database="agg_" + exc.input['database'], name="agg" + exc.input['name'])
            except:
                print(f"Input {exc.input['name']} not found in agg_{exc.input['database']}, skipping...")
                raise
            new_act.new_edge(input=agg_input, amount=exc.amount, type="technosphere").save()


In [ ]:
if "agg_grid_expansion_prospective" in bd.databases:
    del bd.databases["agg_grid_expansion_prospective"]

# Clean up old parameter group if it exists
from bw2data.parameters import Group, ActivityParameter, ParameterizedExchange
agg_param_group = "agg_grid_expansion_prospective_params"
try:
    group = Group.get(name=agg_param_group)
    ActivityParameter.delete().where(ActivityParameter.group == agg_param_group).execute()
    ParameterizedExchange.delete().where(ParameterizedExchange.group == agg_param_group).execute()
    group.delete_instance()
    print(f"Cleaned up old group: {agg_param_group}")
except:
    pass

new_db = bd.Database("agg_grid_expansion_prospective")
new_db.register()

old_expansion_db = bd.Database("grid_expansion_prospective")

for act in old_expansion_db:
    new_act = new_db.new_node(name="agg" + act['name'])  # grid node for a specific year
    new_act.save()
    new_act.new_edge(input=new_act, amount=1, type="production").save()
    
    has_formula = False
    for exc in act.technosphere():
        agg_input = bd.get_node(database="agg_" + exc.input['database'], name="agg" + exc.input['name'])
        
        # Copy the formula if it exists, along with the amount
        if exc.get("formula"):
            new_act.new_edge(
                input=agg_input, 
                amount=exc.amount, 
                type="technosphere",
                formula=exc["formula"]
            ).save()
            has_formula = True
        else:
            new_act.new_edge(
                input=agg_input, 
                amount=exc.amount, 
                type="technosphere"
            ).save()
    
    # Add to parameter group if this activity has any formulas
    if has_formula:
        bd.parameters.add_exchanges_to_group(agg_param_group, new_act)

# Recalculate the aggregated parameter group
try:
    ActivityParameter.recalculate(agg_param_group)
    print(f"Created aggregated grid expansion nodes with parameterized exchanges.")
except Exception as e:
    print(f"Note: Parameter recalculation skipped or failed: {e}")


### Adding uncertainty info

In [ ]:
import numpy as np

In [ ]:
# cleanup
for sub_period in bd.Database("agg_grid_expansion_prospective"):
        for comp_exc in sub_period.technosphere():
            for prop in ["uncertainty type", "loc", "scale", "shape", "minimum", "maximum"]:
                if comp_exc.get(prop):
                    del comp_exc[prop]
            comp_exc.save()
            for mat_exc in comp_exc.input.technosphere():
                for prop in ["uncertainty type", "loc", "scale", "shape", "minimum", "maximum"]:
                    if mat_exc.get(prop):
                        del mat_exc[prop]
                mat_exc.save()
                for bios_exc in mat_exc.input.biosphere():
                    for prop in ["uncertainty type", "loc", "scale", "shape", "minimum", "maximum"]:
                        if bios_exc.get(prop):
                            del bios_exc[prop]
                    bios_exc.save()

In [ ]:
def get_uncertainty_factor(year, base_uncertainty=0.1, final_uncertainty=0.35, start_year=2023, end_year=2045):
        years_total = end_year - start_year
        years_elapsed = year - start_year
        uncertainty_factor = base_uncertainty + (final_uncertainty - base_uncertainty) * (years_elapsed / years_total)
        return uncertainty_factor
    
for sub_period in bd.Database("agg_grid_expansion_prospective"):
    
    if not sub_period["name"].startswith("agggrid_"):
        continue
    
    year = int(sub_period['name'].split("_")[-1])
    for comp_exc in sub_period.technosphere(): # GRID EXPANSION NUMBERS
        comp_exc["uncertainty type"] = 5 # Triangular distribution
        comp_exc["loc"] = comp_exc.amount
        comp_exc["scale"] = np.nan
        comp_exc["shape"] = np.nan
        mini = comp_exc.amount - comp_exc.amount * get_uncertainty_factor(year, base_uncertainty=0.05, final_uncertainty=0.20)
        maxi = comp_exc.amount + comp_exc.amount * get_uncertainty_factor(year, base_uncertainty=0.05, final_uncertainty=0.20)
        if comp_exc.amount < 0:
            mini, maxi = maxi, mini  # swap for negative amounts
        assert mini <= maxi, f"Minimum {mini} is not less than maximum {maxi} for component {comp_exc.input['name']} in year {year}"
        comp_exc["minimum"] = mini
        comp_exc["maximum"] = maxi
        comp_exc.save()
        
        for mat_exc in comp_exc.input.technosphere(): # COMPONENT MATERIAL DEMANDS
            mat_exc["uncertainty type"] = 5 # Triangular distribution
            mat_exc["loc"] = mat_exc.amount
            mat_exc["scale"] = np.nan
            mat_exc["shape"] = np.nan
            mini = mat_exc.amount - mat_exc.amount * get_uncertainty_factor(year, base_uncertainty=0.01, final_uncertainty=0.05)
            maxi = mat_exc.amount + mat_exc.amount * get_uncertainty_factor(year, base_uncertainty=0.01, final_uncertainty=0.05)
            if mat_exc.amount < 0:
                mini, maxi = maxi, mini  # swap for negative amounts
            assert mini <= maxi, f"Minimum {mini} is not less than maximum {maxi} for material {mat_exc.input['name']} in year {year}"
            mat_exc["minimum"] = mini
            mat_exc["maximum"] = maxi
            mat_exc.save()
            
            for emission in mat_exc.input.biosphere(): # MATERIAL GHG EMISSIONS
                emission["uncertainty type"] = 5 # Triangular distribution
                emission["loc"] = emission.amount
                emission["scale"] = np.nan
                emission["shape"] = np.nan
                mini = emission.amount - emission.amount * get_uncertainty_factor(year, base_uncertainty=0.05, final_uncertainty=0.2)
                maxi = emission.amount + emission.amount * get_uncertainty_factor(year, base_uncertainty=0.05, final_uncertainty=0.2)
                if emission.amount < 0:
                    mini, maxi = maxi, mini  # swap for negative amounts
                assert mini <= maxi, f"Minimum {mini} is not less than maximum {maxi} for emission {emission.input['name']} in year {year}"
                emission["minimum"] = mini
                emission["maximum"] = maxi
                emission.save()

Creating aggregated grid expansion nodes over all expansion periods

In [ ]:
for node in db:
    if "expansion" in node['name']:
        node.delete()

exp_npi = db.new_node(name="expansion_NPi")
exp_npi.new_edge(input=exp_npi, amount=1, type="production").save()

exp_pkbudg1000 = db.new_node(name="expansion_PkBudg1000")
exp_pkbudg1000.new_edge(input=exp_pkbudg1000, amount=1, type="production").save()

exp_pkbudg650 = db.new_node(name="expansion_PkBudg650")
exp_pkbudg650.new_edge(input=exp_pkbudg650, amount=1, type="production").save()

for node in db:
    if node['name'].startswith("agggrid_NPi_"):
        exp_npi.new_edge(input=node, amount=1, type="technosphere").save()
    elif node['name'].startswith("agggrid_PkBudg1000_"):
        exp_pkbudg1000.new_edge(input=node, amount=1, type="technosphere").save()
    elif node['name'].startswith("agggrid_PkBudg650_"):
        exp_pkbudg650.new_edge(input=node, amount=1, type="technosphere").save()

exp_npi.save()
exp_pkbudg1000.save()
exp_pkbudg650.save()